In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# invoke方法和stream方法中的Token统计支持

In [2]:
from langchain_zhipu import ChatZhipuAI
from langchain_core.callbacks import BaseCallbackHandler

class CountTokenCallbackHandler(BaseCallbackHandler):
	def on_llm_end(self, response, **kwargs):
		print("-"*20, "Token统计", "-"*20)
		print(response)
		print("-"*20, "--------", "-"*20)

In [3]:
from langchain_zhipu import ChatZhipuAI
llm_glm = ChatZhipuAI(callbacks=[CountTokenCallbackHandler()])

**支持invoke的token统计：**

In [4]:
llm_glm.invoke("你是什么模型？")

-------------------- Token统计 --------------------
generations=[[ChatGeneration(text='我是一个基于语言的人工智能模型，名为 ChatGLM，现在又名智谱清言。我是由清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-130B 开发的，我的任务是针对用户的问题和要求提供适当的答复和支持。我是一种大规模预训练语言模型，基于 Transformer 架构，使用了千亿个参数进行训练，能够进行自然语言处理和生成。', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='我是一个基于语言的人工智能模型，名为 ChatGLM，现在又名智谱清言。我是由清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-130B 开发的，我的任务是针对用户的问题和要求提供适当的答复和支持。我是一种大规模预训练语言模型，基于 Transformer 架构，使用了千亿个参数进行训练，能够进行自然语言处理和生成。'))]] llm_output={'id': '8563447219556116499', 'created': 1712989026, 'token_usage': {'completion_tokens': 92, 'prompt_tokens': 9, 'total_tokens': 101}, 'model_name': 'glm-4'} run=None
-------------------- -------- --------------------


AIMessage(content='我是一个基于语言的人工智能模型，名为 ChatGLM，现在又名智谱清言。我是由清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-130B 开发的，我的任务是针对用户的问题和要求提供适当的答复和支持。我是一种大规模预训练语言模型，基于 Transformer 架构，使用了千亿个参数进行训练，能够进行自然语言处理和生成。')

**支持stream方法的token统计：**

In [5]:
for s in llm_glm.stream([{"role": "user", "content": "你是什么模型？"}]):
  print(s)

content='我' id='8561159891771758252'
content='使用的' id='8561159891771758252'
content='模型' id='8561159891771758252'
content='是' id='8561159891771758252'
content='清华大学' id='8561159891771758252'
content=' K' id='8561159891771758252'
content='EG' id='8561159891771758252'
content=' 实' id='8561159891771758252'
content='验' id='8561159891771758252'
content='室' id='8561159891771758252'
content='和' id='8561159891771758252'
content='智' id='8561159891771758252'
content='谱' id='8561159891771758252'
content='AI' id='8561159891771758252'
content='共同' id='8561159891771758252'
content='训练' id='8561159891771758252'
content='的' id='8561159891771758252'
content=' GL' id='8561159891771758252'
content='M' id='8561159891771758252'
content=' 模' id='8561159891771758252'
content='型' id='8561159891771758252'
content='，' id='8561159891771758252'
content='一种' id='8561159891771758252'
content='基于' id='8561159891771758252'
content=' Transformer' id='8561159891771758252'
content=' 的' id='8561159891771758252'
content='

In [6]:
async for s in llm_glm.astream([{"role": "user", "content": "你是什么模型？"}]):
  print(s)

content='我' id='8563446566721082204'
content='使用的' id='8563446566721082204'
content='模型' id='8563446566721082204'
content='是' id='8563446566721082204'
content='清华大学' id='8563446566721082204'
content=' K' id='8563446566721082204'
content='EG' id='8563446566721082204'
content=' 实' id='8563446566721082204'
content='验' id='8563446566721082204'
content='室' id='8563446566721082204'
content='和' id='8563446566721082204'
content='智' id='8563446566721082204'
content='谱' id='8563446566721082204'
content='AI' id='8563446566721082204'
content='共同' id='8563446566721082204'
content='训练' id='8563446566721082204'
content='的' id='8563446566721082204'
content=' GL' id='8563446566721082204'
content='M' id='8563446566721082204'
content=' 模' id='8563446566721082204'
content='型' id='8563446566721082204'
content='，' id='8563446566721082204'
content='一种' id='8563446566721082204'
content='基于' id='8563446566721082204'
content=' Transformer' id='8563446566721082204'
content=' 的' id='8563446566721082204'
content='

# 智能体中的Token统计

In [12]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub
from langchain_zhipu import ChatZhipuAI

from langchain_core.callbacks import BaseCallbackHandler

class CountTokenCallbackHandler(BaseCallbackHandler):
	def on_llm_end(self, response, **kwargs):
		print("-"*20, "Token统计", "-"*20)
		print(response)
		print("-"*20, "--------", "-"*20)

tools = []
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatZhipuAI(model="glm-3-turbo", temperature=0.1, callbacks=[CountTokenCallbackHandler()])
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

async for chunk in agent_executor.astream_events({"input": "你好"}, version="v1"):
    print(chunk)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-zhipu-Ddey8KS3-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(




> Entering new AgentExecutor chain...
{'event': 'on_chain_start', 'run_id': 'b5ee86af-083e-42c0-8247-5ccb96ba6467', 'name': 'AgentExecutor', 'tags': [], 'metadata': {}, 'data': {'input': {'input': '你好'}}}
{'event': 'on_chain_start', 'name': 'RunnableSequence', 'run_id': 'fdb8797b-5432-47a2-a422-27ff020c69ef', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': 'c28385b0-744b-4d5f-a9e2-67319c7438dc', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {}}
{'event': 'on_chain_stream', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': 'c28385b0-744b-4d5f-a9e2-67319c7438dc', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'chunk': {'input': '你好', 'intermediate_steps': []}}}
{'event': 'on_chain_start', 'name': 'RunnableParallel<agent_scratchpad>', 'run_id': '62bdeaa7-e1d2-4f88-8993-d1370b02e99c', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableLambda', 'run_id': '20bfd19d-015b-4de5-

In [16]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub
from langchain_zhipu import ChatZhipuAI

from langchain_core.callbacks import BaseCallbackHandler

class CountTokenCallbackHandler(BaseCallbackHandler):
	def on_llm_end(self, response, **kwargs):
		print("-"*20, "Token统计", "-"*20)
		print(response)
		print("-"*20, "--------", "-"*20)

tools = []
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatZhipuAI(temperature=0.1, callbacks=[CountTokenCallbackHandler()])
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

async for chunk in agent_executor.astream_events({"input": "你好"}, version="v1"):
    print(chunk)



> Entering new AgentExecutor chain...
{'event': 'on_chain_start', 'run_id': '386293b6-71e8-4bd3-a4d0-2d7aa3a77eaa', 'name': 'AgentExecutor', 'tags': [], 'metadata': {}, 'data': {'input': {'input': '你好'}}}
{'event': 'on_chain_start', 'name': 'RunnableSequence', 'run_id': 'd1b1ac7c-23ed-42ea-b20b-e2e2b211755b', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': '15a37c33-efc2-4315-a6a4-beef5a24482b', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {}}
{'event': 'on_chain_stream', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': '15a37c33-efc2-4315-a6a4-beef5a24482b', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'chunk': {'input': '你好', 'intermediate_steps': []}}}
{'event': 'on_chain_start', 'name': 'RunnableParallel<agent_scratchpad>', 'run_id': '25db4c3a-e86f-4235-ada9-5d805ed471e7', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableLambda', 'run_id': '3e8821ef-829f-4bc3-

# ChatOpenAI的Token统计支持

In [ ]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(callbacks=[CountTokenCallbackHandler()])

**ChatOpenAI支持invoke方法的token统计：**

In [ ]:
llm_gpt.invoke("你是什么模型？")

-------------------- Token统计 --------------------
generations=[[ChatGeneration(text='我是一个基于人工智能技术的语言模型，用于回答用户提出的问题和进行对话。', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='我是一个基于人工智能技术的语言模型，用于回答用户提出的问题和进行对话。'))]] llm_output={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 15, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None} run=None
-------------------- -------- --------------------


AIMessage(content='我是一个基于人工智能技术的语言模型，用于回答用户提出的问题和进行对话。')

**但并不支持stream方法的token统计：**

In [ ]:
for chunk in llm_gpt.stream("你是什么模型？"):
    print(chunk)

content=''
content='我'
content='是'
content='一个'
content='基'
content='于'
content='人'
content='工'
content='智'
content='能'
content='的'
content='聊'
content='天'
content='机'
content='器'
content='人'
content='模'
content='型'
content='。'
content=''
-------------------- Token统计 --------------------
generations=[[ChatGenerationChunk(text='我是一个基于人工智能的聊天机器人模型。', generation_info={'finish_reason': 'stop'}, message=AIMessageChunk(content='我是一个基于人工智能的聊天机器人模型。'))]] llm_output=None run=None
-------------------- -------- --------------------
